In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import nt_search
from Bio.Align.Applications import MafftCommandline
import os
import glob
import random

## loop through multiple fasta and perform codon-based alignment

In [6]:
def translate_cds_to_aa(cds_sequence):
    return cds_sequence.translate()

def map_gaps_to_cds(amino_acid_alignment, original_cds_sequences):
    aligned_cds_sequences = {}

    for aa_record in amino_acid_alignment:
        cds_record = next(cds for cds in original_cds_sequences if cds.id == aa_record.id)

        aligned_cds_seq = ""
        aa_pos = 0

        for aa in str(aa_record.seq):
            if aa == "-":
                aligned_cds_seq += "---"
            else:
                aligned_cds_seq += str(cds_record.seq[aa_pos * 3 : (aa_pos + 1) * 3])
                aa_pos += 1

        aligned_cds_record = SeqRecord(Seq(aligned_cds_seq), id=cds_record.id, description="")
        aligned_cds_sequences[aligned_cds_record.id] = aligned_cds_record

    return list(aligned_cds_sequences.values())

def perform_cds_alignment(input_cds_file, output_cds_alignment_file):
    # Step 1: Translate CDS to Amino Acid Sequences
    amino_acid_sequences = []

    for record in SeqIO.parse(input_cds_file, "fasta"):
        amino_acid_sequence = translate_cds_to_aa(record.seq)
        amino_acid_record = SeqRecord(amino_acid_sequence, id=record.id, description="")
        amino_acid_sequences.append(amino_acid_record)

    SeqIO.write(amino_acid_sequences, "amino_acid_sequences.fasta", "fasta")

    # Step 2: Amino Acid Sequence Alignment
    mafft_cline = MafftCommandline(input="amino_acid_sequences.fasta", auto=True)
    stdout, stderr = mafft_cline()

    with open("amino_acid_alignment.fasta", "w") as f:
        f.write(stdout)

    # Step 3: Convert Amino Acid Alignment to CDS Alignment
    amino_acid_alignment = list(SeqIO.parse("amino_acid_alignment.fasta", "fasta"))
    original_cds_sequences = list(SeqIO.parse(input_cds_file, "fasta"))

    aligned_cds_sequences = map_gaps_to_cds(amino_acid_alignment, original_cds_sequences)

    # Step 4: Write the final CDS alignment to a file
    SeqIO.write(aligned_cds_sequences, output_cds_alignment_file, "fasta")

if __name__ == "__main__":

    # List of FASTA files
    fasta_files = glob.glob("renamed_*.fasta")

    for input_cds_file in fasta_files:
        output_cds_alignment_file = input_cds_file[:-2]
        perform_cds_alignment(input_cds_file, output_cds_alignment_file)

## check cds sequence length

In [7]:
def report_sequences_with_invalid_length(input_cds_file):
    invalid_sequences = []

    for record in SeqIO.parse(input_cds_file, "fasta"):
        if len(record.seq) % 3 != 0:
            invalid_sequences.append(record.id)

    if invalid_sequences:
        print("file=", input_cds_file)
        print(f"Warning: The following CDS sequences have lengths not multiple of three:")
        for seq_id in invalid_sequences:
            print(seq_id)

if __name__ == "__main__":

    # List of FASTA files
    fasta_files = glob.glob("renamed_*.fasta")

    for fasta_file in fasta_files:
        # Check for invalid CDS sequence lengths
        report_sequences_with_invalid_length(fasta_file)

file= renamed_OG0015453.txt.fasta
OryzaSativa


## convert fas alignment to nexus format

In [ ]:
!ls *.fas |while read R;do seqmagick convert --output-format nexus --alphabet dna $R $R".nex";done

## concatenate multiple alignment files

In [33]:
# Specify the desired directory path
new_directory = '/data/igenome/single-copy-OG/renamed_aligned_sequences'

# Change the working directory
os.chdir(new_directory)

# the combine function takes a list of tuples [(name, nexus instance)...],
# if we provide the file names in a list we can use a list comprehension to
# create these tuples
from Bio.Nexus import Nexus
!rm combined.nex
nexi = []
file_list = glob.glob("*.nex")
## random50 = random.sample(file_list, 50)
nexi = [(fname, Nexus.Nexus(fname)) for fname in file_list]

combined = Nexus.combine(nexi)

with open("combined.nex", "w") as f:
    combined.write_nexus_data(filename=f)

rm: cannot remove 'combined.nex': No such file or directory


In [30]:
type(combined)

Bio.Nexus.Nexus.Nexus

In [24]:
# Specify the desired directory path
new_directory = '/data/igenome/single-copy-OG/renamed_aligned_sequences'

# Change the working directory
os.chdir(new_directory)

# the combine function takes a list of tuples [(name, nexus instance)...],
# if we provide the file names in a list we can use a list comprehension to
# create these tuples
from Bio.Nexus import Nexus
nexi = []
file_list = glob.glob("*.nex")

In [25]:
len(file_list)

268

In [26]:
# Define the size of each sublist
sublist_size = 54

# Use list comprehension to create the sublists
sublists = [file_list[i * sublist_size:(i + 1) * sublist_size] for i in range(2)]

# Print the sublists
for i, sublist in enumerate(sublists, start=1):
    file_name = "Group" + str(i) + ".nex"
    nexi = []
    with open(file_name, "w") as f:
        nexi = [(fname, Nexus.Nexus(fname)) for fname in sublist]
        combined = Nexus.combine(nexi)
        combined.write_nexus_data(filename=f)

In [12]:
type(nexi)

list